In [1]:
from read_datasetBreakfast import load_data, read_mapping_dict
import os  
import torch
import numpy as np
import os.path 
import joblib

In [ ]:
def _isArrayLike(obj):
    return hasattr(obj, '__iter__') and hasattr(obj, '__len__')

 
def load_data(split_load, actions_dict, GT_folder, DATA_folder, datatype = 'training',):
    file_ptr = open(split_load, 'r')
    content_all = file_ptr.read().split('\n')[1:-1]
    content_all = [x.strip('./data/groundTruth/') + 't' for x in content_all]
    all_tasks = ['tea', 'cereals', 'coffee', 'friedegg', 'juice', 'milk', 'sandwich', 'scrambledegg', 'pancake', 'salat']
    
    if datatype == 'training':
        data_breakfast = []
        labels_breakfast = []
        for content in content_all:
        
            file_ptr = open( GT_folder + content, 'r')
            curr_gt = file_ptr.read().split('\n')[:-1]
            loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
            curr_data = np.loadtxt(loc_curr_data, dtype='float32')
            label_curr_video = []
            for iik in range(len(curr_gt)):
                label_curr_video.append( actions_dict[curr_gt[iik]] )
         
            data_breakfast.append(torch.tensor(curr_data,  dtype=torch.float64 ) )
            labels_breakfast.append(label_curr_video )
    
        labels_uniq, labels_uniq_loc = get_label_bounds(labels_breakfast)
        print("Finish Load the Training data and labels!!!")     
        return  data_breakfast, labels_uniq
    if datatype == 'test':
        data_breakfast = []
        
        segment = []
        for content in content_all:
            
            #file_ptr = open( GT_folder + content, 'r')
            #curr_gt = file_ptr.read().split('\n')[:-1]
            
            loc_curr_data = DATA_folder + os.path.splitext(content)[0] + '.gz'
        
            curr_data = np.loadtxt(loc_curr_data, dtype='float32')
                                 
            data_breakfast.append(torch.tensor(curr_data,  dtype=torch.float64 ) )
            
        
        return data_breakfast


def get_label_bounds( data_labels):
    labels_uniq = []
    labels_uniq_loc = []
    for kki in range(0, len(data_labels) ):
        uniq_group, indc_group = get_label_length_seq(data_labels[kki])
        labels_uniq.append(uniq_group[1:-1])
        labels_uniq_loc.append(indc_group[1:-1])
    return labels_uniq, labels_uniq_loc

def get_label_length_seq(content):
    label_seq = []
    length_seq = []
    start = 0
    length_seq.append(0)
    for i in range(len(content)):
        if content[i] != content[start]:
            #print(content[i])
            label_seq.append(content[start])
            length_seq.append(i)
            start = i
    label_seq.append(content[start])
    length_seq.append(len(content))
    
    if content[-1] != 0:
        label_seq.append(content[-1])
    
    return label_seq, length_seq


def get_maxpool_lstm_data(cData, indices):
    list_data = []
    for kkl in range(len(indices)-1):
        cur_start = indices[kkl]
        cur_end = indices[kkl+1]
        if cur_end > cur_start:
            list_data.append(torch.max(cData[cur_start:cur_end,:],
                                       0)[0].squeeze(0))
        else:
            list_data.append(torch.max(cData[cur_start:cur_end+1,:],
                                       0)[0].squeeze(0))
    list_data  =  torch.stack(list_data)
    return list_data

def read_mapping_dict(mapping_file):
    file_ptr = open(mapping_file, 'r')
    actions = file_ptr.read().split('\n')[:-1]

    actions_dict=dict()
    for a in actions:
        actions_dict[a.split()[1]] = int(a.split()[0])

    return actions_dict

In [2]:
COMP_PATH = ''
train_split =  os.path.join(COMP_PATH, 'splits/train.split1.bundle') #Train Split
test_split  =  os.path.join(COMP_PATH, 'splits/test.split1.bundle') #Test Split
GT_folder   =  os.path.join(COMP_PATH, 'groundTruth/') #Ground Truth Labels for each training video 
DATA_folder =  os.path.join(COMP_PATH, 'data/') #Frame I3D features for all videos
mapping_loc =  os.path.join(COMP_PATH, 'splits/mapping_bf.txt') 

In [3]:
actions_dict = read_mapping_dict(mapping_loc)

In [4]:
data_feat = load_data( test_split, actions_dict, GT_folder, DATA_folder, datatype = 'test') #Get features only

In [5]:
joblib.dump(data_feat,  'test_feature.joblib')

['test_feature.joblib']

In [6]:
train_feat, train_labels = load_data( train_split, actions_dict, GT_folder, DATA_folder, datatype = 'training') #Get features and labels

Finish Load the Training data and labels!!!


In [7]:
import joblib
joblib.dump(train_feat,  'train_feature.joblib')

['train_feature.joblib']

In [8]:
import joblib
joblib.dump(train_labels,  'train_label.joblib')

['train_label.joblib']